## Fairness LLM
### Metadata Curator

In [ ]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()
parent_dir = os.path.dirname(os.getcwd())
base_dir = os.getcwd() + '/fairness_llm'
sys.path.append(parent_dir)
from download import download_dataset
from section.fairness.fairness_llm.pipeline import main

### Download and build the dataset_pool

In [ ]:
download_dataset(
    section='fairness_llm',
    output_path=base_dir+'/metadata'
)

In [ ]:
main(base_dir=base_dir)

### Retrieve and build the dynamic dataset

In [ ]:
main(
    base_dir=base_dir,
    mode='dynamic'
)

### Contexual Variator

In [ ]:
from src.contextual_variator import contextual_variator_cli

contextual_variator_cli(
    dataset_folder=base_dir + '/final'
)

### Response Generator

In [ ]:
from src.response_generator.lm_responses import process_data_folder

request_type = ['llm']
data_folder = base_dir + '/final'
# async_list=['gpt-4o', 'gpt-4o-mini',"gpt-3.5-turbo", 'claude-3.5-sonnet', 'claude-3-haiku', 'gemini-1.5-pro',       
#             'gemini-1.5-flash', 'gemma-2-27B', 'llama-3.1-70B','llama-3.1-8B', 'glm-4-plus', 'qwen-2.5-72B', 
#             'mistral-8x7B' ,"mistral-8x22B", "yi-lightning", 'deepseek-chat']

async_list = ['gpt-4o-mini', 'llama-3.1-8B']
sync_list = []

prompt_key = 'prompt'
result_key = 'responses'
file_name_extension = '_responses'

await process_data_folder(
    data_folder=data_folder,
    request_type=request_type,
    async_list=async_list,
    sync_list=sync_list,
    prompt_key=prompt_key,
    result_key=result_key,
    file_name_extension=file_name_extension,
)

### Judge Processor

In [ ]:
from src.evaluation import judge_response

data_folder = base_dir + '/final'

target_models = ['llama-3.1-8B', 'gpt-4o-mini']

judge_type = 'llm'
judge_key = 'toxicity' if judge_type == 'toxicity' else 'judge'
response_key = ['responses']

async_judge_model = ['gpt-4o-mini']

config_path = os.path.join(parent_dir, 'src/config/judge_prompt.yaml')

await judge_response(
    data_folder=data_folder,
    async_judge_model=async_judge_model,
    target_models=target_models,
    judge_type=judge_type,
    config_path=config_path,
    response_key=response_key,
    judge_key=judge_key,
    response_extension='_responses',
    judge_extension='_judge',
    reverse_choice=False
)

In [ ]:
from section.truthfulness.truthfulness_llm.evaluate import run, merge_results
print(base_dir)
run(base_dir + '/final/')
merge_results(base_dir + '/final/')